# Aprendizagem Supervisionado

In [ ]:

import pandas as pd #bibliotecapara manipulação de dados
import numpy as np  #biblioteca quer ajuda em cálculos

In [ ]:
#Leitura da base de dados tratada/limpa
df = pd.read_csv("PS_trans.csv")
df.head()


hora tipo_transacao     valor  saldo_origem_antigo  saldo_origem_novo  \
0     0        PAYMENT   9839.64             170136.0          160296.36   
1     0        PAYMENT   1864.28              21249.0           19384.72   
2     0       TRANSFER    181.00                181.0               0.00   
3     0       CASH_OUT    181.00                181.0               0.00   
4     0        PAYMENT  11668.14              41554.0           29885.86   

   saldo_destino_antigo  saldo_destino_novo  eh_fraude tipo_destino  
0                   0.0                 0.0          0            M  
1                   0.0                 0.0          0            M  
2                   0.0                 0.0          1            C  
3               21182.0                 0.0          1            C  
4                   0.0                 0.0          0            M

In [ ]:
df.isnull().sum()


hora                    0
tipo_transacao          0
valor                   0
saldo_origem_antigo     0
saldo_origem_novo       0
saldo_destino_antigo    0
saldo_destino_novo      0
eh_fraude               0
tipo_destino            0
dtype: int64

## tranformando valores categóricos para numéricos:
Criando uma copia para não auterar fonte, contudo fazendo a tranformação numerica para melhor entendimento da maquina.
tipo_transacao
    
    'CASH_IN': 0,
    'CASH_OUT': 1,
    'DEBITO': 2,
    'PAGAMENTO': 3,
    'TRANSFERENCIA': 4


 tipo_destino

    'C': 0,
    'M': 1

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Criar cópia do dataframe para preservar os dados originais
df_encoded = df.copy()

# 1. Codificar tipo_transacao
encoder_transacao = LabelEncoder()
df_encoded['tipo_transacao'] = encoder_transacao.fit_transform(df_encoded['tipo_transacao'])

# Obter mapeamento
mapeamento_transacao = dict(zip(encoder_transacao.classes_, encoder_transacao.transform(encoder_transacao.classes_)))
print("\nMapeamento para 'tipo_transacao':", mapeamento_transacao)

# 2. Codificar tipo_destino
encoder_destino = LabelEncoder()
df_encoded['tipo_destino'] = encoder_destino.fit_transform(df_encoded['tipo_destino'].astype(str))  # Garantir que seja string

# Obter mapeamento
mapeamento_destino = dict(zip(encoder_destino.classes_, encoder_destino.transform(encoder_destino.classes_)))
print("Mapeamento para 'tipo_destino':", mapeamento_destino)

# 3. Tratar valores NaN restantes
df_encoded = df_encoded.fillna({
    'valor': 0,
    'saldo_destino_antigo': 0,
    'saldo_destino_novo': 0
})
df_encoded = df_encoded.fillna(0)  # Preenche quaisquer NaNs restantes no dataframe

# 4. Verificar resultado
print("\nDataFrame após transformação:")
print(df_encoded[['tipo_transacao', 'tipo_destino']].head())

print("\nTipos de dados finais:")
print(df_encoded.dtypes)



Mapeamento para 'tipo_transacao': {'CASH_IN': np.int64(0), 'CASH_OUT': np.int64(1), 'DEBIT': np.int64(2), 'PAYMENT': np.int64(3), 'TRANSFER': np.int64(4)}
Mapeamento para 'tipo_destino': {'C': np.int64(0), 'M': np.int64(1)}

DataFrame após transformação:
   tipo_transacao  tipo_destino
0               3             1
1               3             1
2               4             0
3               1             0
4               3             1

Tipos de dados finais:
hora                      int64
tipo_transacao            int64
valor                   float64
saldo_origem_antigo     float64
saldo_origem_novo       float64
saldo_destino_antigo    float64
saldo_destino_novo      float64
eh_fraude                 int64
tipo_destino              int64
dtype: object


In [22]:
print(df_encoded)

         hora  tipo_transacao      valor  saldo_origem_antigo  \
0           0               3    9839.64             170136.0   
1           0               3    1864.28              21249.0   
2           0               4     181.00                181.0   
3           0               1     181.00                181.0   
4           0               3   11668.14              41554.0   
...       ...             ...        ...                  ...   
5206700     0               1  307859.57              58819.0   
5206701     0               1  259841.66             338636.0   
5206702     0               3    8041.75              12543.0   
5206703     0               0   20718.30             185886.0   
5206704     0               3   23051.57               6057.0   

         saldo_origem_novo  saldo_destino_antigo  saldo_destino_novo  \
0                160296.36                  0.00                0.00   
1                 19384.72                  0.00                0.00   
2  

### Processo de Partição de Dados:
Definindo as Features e label para testes e treinos.

In [ ]:
X = df_encoded.drop('eh_fraude', axis=1)
y = df_encoded['eh_fraude']
# Importar Método para Separar os dados em treino e teste
from sklearn.model_selection import train_test_split
# Aui estamos definindo 20% dos dados para teste e  80% para treino
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4165364, 8)
(1041341, 8)
(4165364,)
(1041341,)


In [ ]:
print(y_train.value_counts())


eh_fraude
0    4162055
1       3309
Name: count, dtype: int64


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

# === Modelo 1: Regressão Logística ===
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
lr_preds = lr_model.predict(X_test)

# === Modelo 2: Árvore de Decisão ===
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
dt_preds = dt_model.predict(X_test)

# === Avaliação dos modelos ===
print("=== Regressão Logística ===")
print(classification_report(y_test, lr_preds))
print(confusion_matrix(y_test, lr_preds))

print("\n=== Árvore de Decisão ===")
print(classification_report(y_test, dt_preds))
print(confusion_matrix(y_test, dt_preds))


=== Regressão Logística ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1040529
           1       0.89      0.45      0.60       812

    accuracy                           1.00   1041341
   macro avg       0.94      0.73      0.80   1041341
weighted avg       1.00      1.00      1.00   1041341

[[1040483      46]
 [    444     368]]

=== Árvore de Decisão ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1040529
           1       0.85      0.88      0.86       812

    accuracy                           1.00   1041341
   macro avg       0.92      0.94      0.93   1041341
weighted avg       1.00      1.00      1.00   1041341

[[1040402     127]
 [    101     711]]


### Conclusão:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
# Importar accuracy_score juntamente com as outras métricas
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Listas para armazenar os resultados
acuracia_logistica = []
acuracia_arvore = []

# Repetições para validar robustez
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Modelo 1: Regressão Logística
    modelo_log = LogisticRegression(max_iter=1000)
    modelo_log.fit(X_train, y_train)
    pred_log = modelo_log.predict(X_test)
    acuracia_log = accuracy_score(y_test, pred_log)
    acuracia_logistica.append(acuracia_log)

    # Modelo 2: Árvore de Decisão
    modelo_tree = DecisionTreeClassifier(random_state=42)
    modelo_tree.fit(X_train, y_train)
    pred_tree = modelo_tree.predict(X_test)
    acuracia_tree = accuracy_score(y_test, pred_tree)
    acuracia_arvore.append(acuracia_tree)



Acurácia Média - Regressão Logística: 0.9994907207789443
Acurácia Média - Árvore de Decisão: 0.9997754177866167


In [ ]:
# Resultados finais
print("Acurácia Média - Regressão Logística:", np.mean(acuracia_logistica))
print("Acurácia Média - Árvore de Decisão:", np.mean(acuracia_arvore))

Acurácia Média - Regressão Logística: 0.9994907207789443
Acurácia Média - Árvore de Decisão: 0.9997754177866167


In [38]:
import pickle

# Supondo que o modelo já foi treinado:
modelo_log = LogisticRegression(max_iter=1000)
modelo_log.fit(X_train, y_train)

# Salvar o modelo treinado em um arquivo .pkl
with open('../IA_compet/modelo_regressao_logistica.pkl', 'wb') as arquivo:
    pickle.dump(modelo_log, arquivo)

print("✅ Modelo salvo com sucesso em: ../IA_compet/modelo_regressao_logistica.pkl")



✅ Modelo salvo com sucesso em: ../IA_compet/modelo_regressao_logistica.pkl


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [24]:
%%capture
!pip install gradio  # Instalação sem nenhuma saída
import gradio as gr   # Importação normal


In [39]:

import pickle
import pandas as pd

# Carregar o modelo
with open('../IA_compet/modelo_regressao_logistica.pkl', 'rb') as f:
    modelo = pickle.load(f)

# Colunas usadas durante o treinamento
colunas_modelo = ['hora', 'tipo_transacao', 'valor',
                  'saldo_origem_antigo', 'saldo_origem_novo', 'saldo_destino_antigo', 'saldo_destino_novo', 'tipo_destino']